In [1]:

from datasets import load_dataset

import argparse

import chromadb
from sentence_transformers import SentenceTransformer

import os
import sys
import torch
project_root = os.path.abspath(os.path.join(os.path.dirname('./'), '..'))
print(project_root)
# 添加项目根目录到sys.path
 
sys.path.append(project_root)
from src.utils import load_beir_datasets, load_models,load_json
import argparse



/data/sunmengjie/miniconda3/envs/wmrag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/data/sunmengjie/lpz/ragwm


In [23]:
# ChromadbPath = '/data/sunmengjie/chromdb/chromadb_data_copy/chromadb_data'
# ChromadbPath = '/data/sunmengjie/chromdb/chromadb_data'
# ChromadbPath = '/data/sunmengjie/chromdb/ms_data'
# nfcorpus
ChromadbPath = '/data/sunmengjie/chromdb/wh_db/nfcorpus_db'

In [24]:
from rag.vectorstore_wh import  VectorStore, check_collection

import torch

In [25]:
chroma_client = chromadb.PersistentClient(path=ChromadbPath ) 
collections =  chroma_client.list_collections()
for collection in collections:
     print(collection)
    # print(collection.name)

Collection(name=nfcorpus_contriever_ip)
Collection(name=nfcorpus_contriever_l2)
Collection(name=nfcorpus_ance_cosine)
Collection(name=nfcorpus_contriever-msmarco_cosine)
Collection(name=nfcorpus_contriever_cosine)


In [17]:

class VectorStore:
    def __init__(self, embedding_model, tokenizer, get_emb, dataset, device, collection_name, use_local, distance= 'cosine'):
 
        # self.chroma_client = chromadb.Client()
        # self.use_local = use_local
        ## will autoload and autosave update
        self.chroma_client = chromadb.PersistentClient(path=ChromadbPath ) 
        collections = self.chroma_client.list_collections()
        print(collections)
        collection_exists = any(col.name == collection_name for col in collections)
        if collection_exists and use_local:
            print(f"Here are using an exsiting local chromadb named {collection_name}!!! ")
            self.collection = self.chroma_client.get_collection(name=collection_name)
            print(f"Here are using an exsiting local chromadb named {collection_name}!!! ")
        else:
            print('ok',collection_exists)
            if collection_exists : 
                print(f"Here are delete a existent local chromadb named {collection_name}!!!")
                # collection = self.chroma_client.get_collection(name=collection_name)
                self.chroma_client.delete_collection(name=collection_name)
            print(f"Here are creating a nonexistent local chromadb named {collection_name}!!!")
            self.collection = self.chroma_client.create_collection(name=collection_name, metadata={"hnsw:space": distance})
        
        # Filter the dataset to only include entries with the 'closed_qa' category
        self.embedding_model = embedding_model
        self.tokenizer = tokenizer
        self.dataset = dataset
        self.get_emb = get_emb
        # torch.cuda.set_device(2)
        # device = 'cuda'
        self.device = device
        self.embedding_model.eval()
        self.embedding_model.to(self.device)

    def get_embedding (self, text):
            text_input = self.tokenizer(text,padding=True, truncation=True, return_tensors="pt")
            # text_input = {key: value.cuda() for key, value in text_input.items()}
            text_input = {key: value.to(self.device) for key, value in text_input.items()}

            # print(f"text_input device: {text_input['input_ids'].device}")
            # print(f"embedding_model device: {next(self.embedding_model.parameters()).device}")
            with torch.no_grad():
                text_emb = self.get_emb(self.embedding_model, text_input).squeeze().tolist()
            return text_emb

    # Method to populate the vector store with embeddings from a dataset
    def populate_vectors_backup(self):
        # {'doc0':{'text': "In accounting, minority interest (or non-controlling interest) is the portion of a subsidiary corporation's stock that is not owned by the parent corporation. The magnitude of the minority interest in the subsidiary company is generally less than 50% of outstanding shares, or the corporation would generally cease to be a subsidiary of the parent.[1]", 'title': 'Minority interest'}}
        count = 0 
        for key, value in  self.dataset.items() :
  
            # print(key, value)
            text_emb = self.get_embedding(value['text'])
            self.collection.add(embeddings=[text_emb ], documents=[value['text']], ids=[f'id_{count}'], metadatas={'title': value['title'], 'id':key , 'change':False})
            count+=1

            if count % 100 == 0:
                print(count)

    def populate_vectors(self):
            # 按 key 排序遍历
        
        current_key  =  self.collection.count()
        for count, key in enumerate(sorted(self.dataset.keys())):
            # logger.info(f'i: {i}, key:{key}')
            if count < current_key:
                # 跳过已处理的 key
                # print(f'key has run,count:{count} key:{key}, current_key:{current_key}')
                continue
                        # print(key, value)
            # print(f'key has run,count:{count} key:{key}, current_key:{current_key}')
            # print(self.dataset[key])
            value = self.dataset[key]
            text_emb = self.get_embedding(value['text'])
            self.collection.add(embeddings=[text_emb ], documents=[value['text']], ids=[f'id_{count}'], metadatas={'title': value['title'], 'id':key , 'change':False})
             

            if count % 500 == 0:
                print(count)


    # Method to search the ChromaDB collection for relevant context based on a query
    def search_context(self, query, n_results ):
        text_emb = self.get_embedding(query)
        return self.collection.query(query_embeddings=text_emb, n_results=n_results)

 
    def update_context(self,  id, str_add=''):
        id_data = self.get_id(id)
        
        if id_data['metadatas'][0]['change'] == True and str_add=='':
            # for clean_collection
            context = self.dataset[id_data['metadatas'][0]['id']]['text']
            flag = False  
        elif id_data['metadatas'][0]['change'] == False and str_add=='':
            return True
        else:
            context = id_data['documents'][0] + ' ' + str_add
            flag = True

        embeddings = self.get_embedding(context)
        metadata={'title': self.dataset[id_data['metadatas'][0]['id']]['title'], 'id':id_data['metadatas'][0]['id'], 'change':flag}

        self.collection.update(ids=[id], embeddings=[embeddings], documents =[context], metadatas=metadata)
        # self.collection.update(ids=[id], embeddings=[embeddings], documents =[context], metadatas=metadata)

    def clean_collect(self):
        print('befor clean')
        # 过滤出 metadata 中 'change' 为 True 的记录
        results = self.collection.get(
            where={'change': True},  # 过滤条件
            include=['ids'],  # 包含你想要的数据
            limit=100
        )
        if len(results['ids']) == 0 : return True
        for i in range(len(results['ids'])):
            # id_{count}
            
            doc_id = results['ids'][i]
            count = doc_id.split('_')[1]
            print('befor clean',  self.collection.count())
            if int(count) >= len(self.dataset):
                self.collection.delete(doc_id)
                print('after delete',  self.collection.count())
            else:
                self.update_context(doc_id)
            print(doc_id)
        return True
    # Method to show the contents of the collection
    def show_context(self):
        # top10_documents = self.collection.peek()
        # print(top10_documents)
        # print(self.collection.count())
        id = 'doc0'
        get_id = self.collection.get(ids=[id])
        print(get_id)

    def get_id(self,id):
        # print(self.collection.get(ids=[id]))
        return self.collection.get(ids=[id])    
    
    def inject_direct(self,text):

        text_emb = self.get_embedding(text)
        doc_id = self.collection.count()
        print('before inject direct',doc_id)
        self.collection.add(embeddings=[text_emb ], documents=[text], ids=[f'id_{doc_id}'], metadatas={'title': ' ', 'id':' ' , 'change':True})
        print('after inject direct',self.collection.count())
        return f'id_{doc_id}'
             

    def clean_vectors(self):
            # 按 key 排序遍历
        
        total_count  =  self.collection.count()
        for count in range(total_count):

            ids=f'id_{count}'
            print('befor clean',  self.collection.count())
            if int(count) >= len(self.dataset):
                self.collection.delete(ids)
                print('after delete',  self.collection.count())
            else:
                self.update_context(ids)
            print(ids)

         

In [18]:
import chromadb
def check_collection(collection_name):
        chroma_client = chromadb.PersistentClient(path=ChromadbPath ) 
        collections =  chroma_client.list_collections()
        print(collections)
        collection_exists = any(col.name == collection_name for col in collections)
        total_items = 0
        if collection_exists :
            # print(f" collection exist:{collection_name}")
            collection = chroma_client.get_collection(name=collection_name)
            # # 获取所有 ID
            # all_ids = collection.get(include=['documents' ] )
            # # print(all_ids)
            # print(type(all_ids))
            # print(len(all_ids['ids']))
            # 计算 ID 的数量
            # total_items = len(all_ids['documents'])
            total_items = collection.count()
            print(f"Collection 总数: {total_items}")
            return True, total_items
        else:
            print(f" collection not exist:{collection_name}")
            return False, total_items
        
def delete_collection(collection_name):
        chroma_client = chromadb.PersistentClient(path=ChromadbPath ) 
        collections =  chroma_client.list_collections()
        print(collections)
        collection_exists = any(col.name == collection_name for col in collections)
        total_items = 0
        if collection_exists :
            # print(f" collection exist:{collection_name}")
            collection = chroma_client.get_collection(name=collection_name)
            # # 获取所有 ID
            # all_ids = collection.get(include=['documents' ] )
            # # print(all_ids)
            # print(type(all_ids))
            # print(len(all_ids['ids']))
            # 计算 ID 的数量
            # total_items = len(all_ids['documents'])
            total_items = collection.count()
            print(f"Collection 总数: {total_items}")
            chroma_client.delete_collection(name=collection_name)


In [19]:
torch.cuda.set_device(2)
device = 'cuda'

In [20]:
dataname_list = ['trec-covid','msmarco']
eval_dataset = 'trec-covid'
eval_model_code = "contriever"
score_function ='cosine'
split = 'test'
collection_name = 'trec-covid_contriever_cosine_cp'

In [21]:
model, c_model, tokenizer, get_emb = load_models( eval_model_code)

if eval_dataset == 'msmarco':
    corpus, queries, qrels = load_beir_datasets('msmarco', 'train')
else:
    corpus, queries, qrels = load_beir_datasets(eval_dataset, split)

vectorstore = VectorStore(model, tokenizer, get_emb, corpus, device, collection_name, use_local=True)  


Some weights of the model checkpoint at facebook/contriever were not used when initializing Contriever: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing Contriever from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Contriever from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/data/sunmengjie/miniconda3/envs/wmrag/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/3

/data/sunmengjie/lpz/ragwm/datasets/trec-covid
/data/sunmengjie/lpz/ragwm/datasets/trec-covid


100%|██████████| 171332/171332 [00:01<00:00, 115315.16it/s]


[Collection(name=nfcorpus_contriever_cosine), Collection(name=trec-covid_contriever_cosine), Collection(name=nfcorpus_contriever_ip), Collection(name=trec-covid_contriever_cosine_cp), Collection(name=msmarco_contriever-msmarco_cosine), Collection(name=trec-covid_contriever_ip), Collection(name=trec-covid_contriever_l2), Collection(name=trec-covid_ance_cosine), Collection(name=trec-covid_contriever-msmarco_cosine), Collection(name=msmarco_contriever_cosine), Collection(name=nfcorpus_ance_cosine), Collection(name=msmarco_ance_cosine), Collection(name=nfcorpus_contriever-msmarco_cosine), Collection(name=msmarco_contriever_l2), Collection(name=msmarco_contriever_ip), Collection(name=nfcorpus_contriever_l2), Collection(name=hotpotqa_contriever_cosine)]
Here are using an exsiting local chromadb named trec-covid_contriever_cosine_cp!!! 
Here are using an exsiting local chromadb named trec-covid_contriever_cosine_cp!!! 


In [ ]:
eval_dataset = 'msmarco'
eval_model_code = "contriever"
score_function ='cosine'
split = 'test'
collection_name = 'msmarco_contriever_cosine'

In [22]:
# 清洗数据库
eval_dataset = 'nfcorpus'
eval_model_code = "contriever"
score_function ='cosine'
collection_name = eval_dataset+'_'+eval_model_code+'_'+score_function
split='test'

model, c_model, tokenizer, get_emb = load_models(eval_model_code)

if eval_dataset == 'msmarco':
    corpus, queries, qrels = load_beir_datasets('msmarco', 'train')
else:
    corpus, queries, qrels = load_beir_datasets(eval_dataset, split)

vectorstore = VectorStore(model, tokenizer, get_emb, corpus, device, collection_name, use_local=True)  
check_collection(collection_name) 

Some weights of the model checkpoint at facebook/contriever were not used when initializing Contriever: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing Contriever from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Contriever from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


/data/sunmengjie/lpz/ragwm/datasets/nfcorpus
/data/sunmengjie/lpz/ragwm/datasets/nfcorpus


100%|██████████| 3633/3633 [00:00<00:00, 100222.35it/s]

[Collection(name=nfcorpus_contriever_cosine), Collection(name=trec-covid_contriever_cosine), Collection(name=nfcorpus_contriever_ip), Collection(name=trec-covid_contriever_cosine_cp), Collection(name=msmarco_contriever-msmarco_cosine), Collection(name=trec-covid_contriever_ip), Collection(name=trec-covid_contriever_l2), Collection(name=trec-covid_ance_cosine), Collection(name=trec-covid_contriever-msmarco_cosine), Collection(name=msmarco_contriever_cosine), Collection(name=nfcorpus_ance_cosine), Collection(name=msmarco_ance_cosine), Collection(name=nfcorpus_contriever-msmarco_cosine), Collection(name=msmarco_contriever_l2), Collection(name=msmarco_contriever_ip), Collection(name=nfcorpus_contriever_l2), Collection(name=hotpotqa_contriever_cosine)]
Here are using an exsiting local chromadb named nfcorpus_contriever_cosine!!! 
Here are using an exsiting local chromadb named nfcorpus_contriever_cosine!!! 


[Collection(name=nfcorpus_contriever_cosine), Collection(name=trec-covid_contriever_cosine), Collection(name=nfcorpus_contriever_ip), Collection(name=trec-covid_contriever_cosine_cp), Collection(name=msmarco_contriever-msmarco_cosine), Collection(name=trec-covid_contriever_ip), Collection(name=trec-covid_contriever_l2), Collection(name=trec-covid_ance_cosine), Collection(name=trec-covid_contriever-msmarco_cosine), Collection(name=msmarco_contriever_cosine), Collection(name=nfcorpus_ance_cosine), Collection(name=msmarco_ance_cosine), Collection(name=nfcorpus_contriever-msmarco_cosine), Collection(name=msmarco_contriever_l2), Collection(name=msmarco_contriever_ip), Collection(name=nfcorpus_contriever_l2), Collection(name=hotpotqa_contriever_cosine)]
Collection 总数: 3633


(True, 3633)

In [ ]:
res = vectorstore.search_context("test",5)
res

NameError: name 'vectorstore' is not defined

In [11]:
vectorstore.clean_collect()  

befor clean


ValueError: Expected include item to be one of documents, embeddings, metadatas, distances, uris, data, got ids in get.

In [ ]:
delete_collection("msmarco_contriever_l2")

[Collection(id=02592a9c-7d34-4bb1-b8f2-c555ad31957b, name=nfcorpus_contriever_cosine), Collection(id=09e8a601-7112-4246-b1e2-f433080d42e4, name=trec-covid_contriever_cosine), Collection(id=0b76b752-b792-4f01-b8c0-03072827c957, name=nfcorpus_contriever_ip), Collection(id=226359d3-41b2-40da-9bca-ef67308d3d9e, name=trec-covid_contriever_l2), Collection(id=2ba0db51-f725-44ae-af94-c1a0e8f5d4e5, name=msmarco_contriever-msmarco_cosine), Collection(id=31760893-2b13-41d8-9a95-7a2415c51955, name=trec-covid_contriever_ip), Collection(id=60a4bf7f-07ab-4b26-b465-8522ae251f2e, name=trec-covid_ance_cosine), Collection(id=7f8dece5-1c75-49d9-a78f-b402f329a72b, name=trec-covid_contriever-msmarco_cosine), Collection(id=92a402d2-9e3c-4a9e-a2ba-06481e7755d3, name=msmarco_contriever_cosine), Collection(id=a12c39a1-b64e-422d-9cb3-b03abad03d95, name=nfcorpus_ance_cosine), Collection(id=a6f5ad60-34f6-4bb6-9969-dea08343ac67, name=msmarco_ance_cosine), Collection(id=a994dbd1-6a31-4506-9a74-c6a84555e26b, name=nfc

: 